In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from datasets import load_dataset
import plotting

In [2]:
dataset_name = "loris3/stratified_10m_curriculum"
df_random, curriculum_random = plotting.load_data_for_plotting(dataset_name, "random.pt")
df_lognorm, curriculum_lognorm = plotting.load_data_for_plotting(dataset_name, "lognorm.pt")


In [3]:
import pandas as pd

### Perplexity

In [4]:
# %run python compute_difficulty.py loris3/stratified_10m_curriculum_lognorm loris3/stratified_10m_curriculum

In [5]:
df_perplexity = pd.read_parquet(os.path.join("./difficulty_old", os.path.basename(dataset_name)))

In [6]:
df_lognorm["total"].corr(-df_perplexity["perplexity"], method="kendall")

0.06980971193661108

In [7]:
order_influence = df_lognorm["total"].sort_values(ascending=False).index

In [15]:
order_increasing_difficulty_perplexity =df_perplexity.reindex(df_lognorm.index)["perplexity"].sort_values(ascending=True).index

In [9]:
from scipy.stats import spearmanr


In [16]:
r, p_value = spearmanr(order_influence, order_increasing_difficulty_perplexity, nan_policy="omit")
r

0.0680596365649359

In [17]:
p_value

0.0

In [11]:
# missing_indices = [i for i in list(range(0,1040812+1000)) if i not in df_perplexity.index]
# df_missing = pd.DataFrame([pd.NA for _ in missing_indices], index=missing_indices)
# df_missing.columns=["perplexity"]
# df_missing.index.name = "document_id"
# pd.concat([df_perplexity, df_missing])

In [18]:
from transformers import RobertaConfig,AutoConfig
from transformers import RobertaForMaskedLM

In [12]:
df_perplexity.corrwith(df_lognorm, numeric_only=True, drop=True, method="kendall")

Series([], dtype: object)

In [22]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("loris3/stratified_10m_curriculum_random", max_len=512)
tokenizer

RobertaTokenizerFast(name_or_path='loris3/stratified_10m_curriculum_random', vocab_size=52000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [ ]:
# https://github.com/codebyzeb/CLIMB/blob/dc4f13c1bdf94938d468f0926368dee048485cba/src/data_curriculum/difficulty_scorer/perplexity.py#L77
train_subsample_factor = 1
n_gram = 1

from nltk.util import everygrams

def remove_padding_tokens(example_batch):
    batch = {"unpadded_input_ids": []}
    for example in example_batch["input_ids"]:
        batch["unpadded_input_ids"].append(
            [
                str(_id)
                for _id in example
                if _id != self.tokenizer.pad_token_id  # type: ignore
            ]
        )

    return batch

# removing padding tokens from the dataset
dataset = load_dataset("loris3/stratified_10m_curriculum")["train"]
dataset = dataset.map(
    remove_padding_tokens,
    batched=True,
    num_proc=64,
)

tokenized_text = dataset["unpadded_input_ids"]

# We split the tokenized dataset into n-gram that we can feed into the n-gram model
train_data_n_grams = (
    everygrams(sent, max_len=n_gram)
    for sent in tokenized_text[
        0 : dataset.num_rows : train_subsample_factor
    ]
)

train_vocab = [str(val) for val in self.tokenizer.vocab.values()]

self.lm = MLE(self.n_gram)
self.lm.fit(train_data_n_grams, train_vocab)

def _compute_ngram_perplexity(
self, example: Generator[Tuple[str], Any, None]
):
return self.lm.perplexity(example)

IndentationError: expected an indented block after function definition on line 40 (125635135.py, line 43)